<a href="https://colab.research.google.com/github/albercej/zrh/blob/main/trends/Primary%20Market%20Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#!pip install pandas_ta
#!pip install yfinance
#!pip install ta
!pip install yahoo_fin

     |████████████████████████████████| 81 kB 5.3 MB/s 
     |████████████████████████████████| 83 kB 1.6 MB/s 
     |████████████████████████████████| 138 kB 46.2 MB/s 
     |████████████████████████████████| 112 kB 38.5 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=2032eb08c7caf6503641ff21628bb0a0bbfef033a94237c3fe77285ffac47b49
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=8491923c536271270898894b4e7804cd4fe8fb82ef8b1a6687736066da4af4a4
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=70818c912ff264b199f326306ed01d07b3af76b8237e227874f24168fdcbf14b
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [8]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
import random
from datetime import date
import pandas_ta as ta
from ta.volatility import BollingerBands
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yahoo_fin import stock_info as si
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
#import fix_yahoo_finance as yf
import yfinance as yf

In [9]:
def load_historic_data(symbol):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (600 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbol, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbol)
        return None

In [10]:
def calculate_smas(df):
    df['SMA50'] = ta.sma(df["close"], length=50)
    df['SMA200'] = ta.sma(df["close"], length=200)

    return df

In [11]:
def determine_primary_market(df):
    df['BEAR'] = np.where(df['SMA50'] < df["SMA200"], df['close'], np.NaN)
    df['BULL'] = np.where(df['SMA50'] > df["SMA200"], df['close'], np.NaN) 

    return df

In [12]:
def plot_graph(df):
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['close'], line=dict(color="blue", width=1), name="Close"), row = 1, col = 1)
    
    #  Plot SMAs
    fig.add_trace(go.Line(x = df.index, y = df['SMA50'], line=dict(color="green", width=1), name="SMA50"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['SMA200'], line=dict(color="red", width=1), name="SMA200"), row = 1, col = 1)
    
    
    # Primary Markets
    fig.add_trace(go.Line(x = df.index, y = df['BULL'], line=dict(color="green", width=1), name="Bull"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['BEAR'], line=dict(color="red", width=1), name="Bear"), row = 1, col = 1)
    
    fig.update_layout(
        title={'text':'Primary Markets', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()
    

In [13]:
df = load_historic_data('NDX')
df = calculate_smas(df)
df = determine_primary_market(df)
plot_graph(df)

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [14]:
#  Index Calculation
def load_crypto_data(symbols):
    today = datetime.date.today()
    today_str = today.strftime("%Y-%m-%d")
    #  Get last year's data
    start_date = today - (600 * US_BUSINESS_DAY)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
    # Download data from Yahoo Finance
    try:
        df = si.get_data(symbols, start_date=start_date_str, end_date=today_str, index_as_date=False)
        return df
    except:
        print('Error loading stock data for ' + symbols)
        return None

In [15]:
symbols = ['XLM-USD','DOGE-USD','STX-USD']

start = datetime.datetime(2021,5,1)
end = datetime.datetime(2022,4,29)
crypto_df = yf.download(symbols, start=start, end=end)
#print(crypto_df.head())
#
#
#  Convert columns from tuple to strings
crypto_df.columns = ['{}_{}'.format(x[0], x[1]) for x in crypto_df.columns]
#for (columnName, columnData) in crypto_df.iteritems():
#    print('Column Name : ', columnName)
column_names = ['Close_STX-USD', 'Close_DOGE-USD', 'Close_XLM-USD']  
crypto_df['CRYPTO_INDEX']= crypto_df[column_names].sum(axis=1) / len(symbols)
print(crypto_df['CRYPTO_INDEX'])

[*********************100%***********************]  3 of 3 completed
Date
2021-05-01    1.025609
2021-05-02    1.002909
2021-05-03    1.020647
2021-05-04    1.018919
2021-05-05    1.130237
                ...   
2022-04-25    0.487341
2022-04-26    0.450578
2022-04-27    0.458952
2022-04-28    0.457430
2022-04-29    0.440406
Name: CRYPTO_INDEX, Length: 364, dtype: float64


In [16]:
def plot_crypto_index(df):
    fig = make_subplots(rows=1, cols=1)

    #  Plot close price
    fig.add_trace(go.Line(x = df.index, y = df['CRYPTO_INDEX'], line=dict(color="blue", width=2), name="Crypto Index"), row = 1, col = 1)
    
    #  Plot Coins
    fig.add_trace(go.Line(x = df.index, y = df['Close_STX-USD'], line=dict(color="red", width=1), name="STX-USD"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['Close_DOGE-USD'], line=dict(color="yellow", width=1), name="DOGE-USD"), row = 1, col = 1)
    fig.add_trace(go.Line(x = df.index, y = df['Close_XLM-USD'], line=dict(color="purple", width=1), name="XLM-USD"), row = 1, col = 1)
 
    fig.update_layout(
        title={'text':'Primary Markets', 'x':0.5},
        autosize=False,
        width=800,height=400)
    fig.update_yaxes(range=[0,1000000000],secondary_y=True)
    fig.update_yaxes(visible=False, secondary_y=True)  #hide range slider
    
    fig.show()

In [17]:
plot_crypto_index(crypto_df)

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


